In [48]:
from typing_extensions import Self
import numpy as np
import os
import joblib
import torch

class QuantileModelPredictor:
    def __init__(self, model_directory):
        # Load the scaler and ensure it outputs float64
        self.scaler = joblib.load(os.path.join(model_directory, 'scaler.pkl'))

        self.weights = []
        self.biases = []

        # Load quantiles and their corresponding model weights and biases
        quantiles = sorted(np.load(os.path.join(model_directory, 'quantiles.npy'), allow_pickle=True))
        self.quantiles = quantiles

        for q in self.quantiles:
            model_path = os.path.join(model_directory, f'model_quantile_{q:.2f}.pth')
            model_info = torch.load(model_path)
            # Initialize lists to hold weights and biases for the quantile model
            model_weights = []
            model_biases = []

            # Extract weights and biases from model state dictionary and convert them to float64
            for name, param in model_info['state_dict'].items():
                if 'weight' in name:
                    model_weights.append(param.detach().numpy().astype(np.float64))
                elif 'bias' in name:
                    model_biases.append(param.detach().numpy().astype(np.float64))

            self.weights.append(model_weights)
            self.biases.append(model_biases)


    def predict_quantiles(self, inputs):
      inputs = np.array(inputs, dtype=np.float64).reshape(1, -1)
      scaled_inputs = self.scaler.transform(inputs).astype(np.float64)

      manual_outputs = []
      for i in range(len(self.quantiles)):
          x = scaled_inputs
          layers = len(self.weights[i])
          for j in range(layers):
              weights = self.weights[i][j]
              biases = self.biases[i][j]
              x = np.dot(x, weights.T) + biases
              if j < layers - 1:  # Apply ReLU to all but the last layer
                  x = np.maximum(0, x)
          manual_outputs.append(x.flatten()[0])

      return manual_outputs

# Usage example:
model_directory = '/home/yui/Downloads/read_and_play/model_output'  # Adjust the path as necessary
predictor = QuantileModelPredictor(model_directory)
inputs = [5000, 1000, 15000, 2000,5]  # Example inputs
manual_outputs = predictor.predict_quantiles(inputs)
print("qunatiles:", predictor.quantiles)
print("Manual Outputs:", manual_outputs)

qunatiles: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
Manual Outputs: [1769072883.8054156, 1898413458.9052222, 1986520413.1674604, 2082968245.2450662, 2181855837.155283, 2588593147.505301, 3084116625.7835684, 3435975936.246647, 3941963963.7067432]


In [41]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch
import joblib
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch
import joblib
import os


def optimize(inputs, model_directory):
    model = gp.Model("Quantile_Optimization")
    x_vars = model.addVars(len(inputs) - 1, lb=0, ub=10000, name="Inputs")
    model.update()

    # Set objective as a placeholder to ensure the model is solvable
    model.setObjective(1, GRB.MAXIMIZE)

    # Add constraint that the sum of inputs should not exceed a certain value
    for i in range(len(inputs)-1):
        model.addConstr(x_vars[i] <= 15000, f"MaxInput{i}")

    # Optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Access optimized input values only after optimization
        optimized_inputs = [x_vars[i].X for i in range(len(inputs) - 1)] + [inputs[-1]]


        predictions = predictor.predict_quantiles(optimized_inputs)

        # Compute weighted sum of outputs as the true objective
        weights = np.linspace(1, 2, len(predictions))  # Increasing weights as an example
        weighted_sum = sum(w * p for w, p in zip(weights, predictions))

        print("Optimized inputs:", optimized_inputs)
        print("Weighted sum of predictions:", weighted_sum)
    else:
        print("Optimization was not successful.")

# Example usage
inputs = [1000, 2000, 3000, 4000, 5]  # initial guesses for inputs, last one is fixed
model_directory = '/home/yui/Downloads/read_and_play/model_output'  # Adjust as necessary
optimize(inputs, model_directory)



Defaulting to user installation because normal site-packages is not writeable
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x9919ad91
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [2e+04, 2e+04]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+00
Optimized inputs: [0.0, 0.0, 0.0, 0.0, 5]
Weighted sum of predictions: 87605111900.53123


In [24]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch
import joblib
import os

class QuantileModelPredictor:
    def __init__(self, model_directory):
        self.scaler = joblib.load(os.path.join(model_directory, 'scaler.pkl'))
        quantiles = np.load(os.path.join(model_directory, 'quantiles.npy'), allow_pickle=True)
        self.models = []
        for q in sorted(quantiles):
            model_path = os.path.join(model_directory, f'model_quantile_{q:.2f}.pth')
            model_info = torch.load(model_path)
            self.models.append((model_info['weights'], model_info['state_dict']['output_layer.bias'].numpy()))

    def predict(self, inputs):
        inputs_scaled = self.scaler.transform(np.array([inputs]))
        outputs = []
        for weights, bias in self.models:
            output = inputs_scaled
            for w, b in zip(weights, bias):
                output = np.dot(output, w.T) + b
                output = np.maximum(output, 0)  # ReLU activation
            outputs.append(output.flatten()[-1])
        return outputs

def optimize(inputs, model_directory, n_trials=10):
    best_weighted_sum = -np.inf
    best_inputs = None

    for _ in range(n_trials):
        model = gp.Model("Quantile_Optimization")
        x_vars = model.addVars(len(inputs) - 1, lb=0, ub=15000, name="Inputs", vtype=GRB.CONTINUOUS)
        model.update()

        # Randomize starting points
        for var in x_vars.values():
            var.Start = np.random.uniform(0, 15000)

        # Set up the predictor
        predictor = QuantileModelPredictor(model_directory)

        # Define objective function using lazy constraints
        def callback(model, where):
            if where == GRB.Callback.MIPSOL:
                inp = model.cbGetSolution([model._vars[i] for i in range(len(inputs) - 1)]) + [inputs[-1]]
                predictions = predictor.predict(inp)
                weights = np.linspace(1, 2, len(predictions))
                objective_value = sum(w * p for w, p in zip(weights, predictions))
                model.cbLazy(objective_value >= model.cbGet(GRB.Callback.MIPSOL_OBJBST))

        model._vars = x_vars
        model.optimize(callback)

        if model.status == GRB.OPTIMAL:
            optimized_inputs = [x_vars[i].X for i in range(len(inputs) - 1)] + [inputs[-1]]
            predictions = predictor.predict(optimized_inputs)
            weights = np.linspace(1, 2, len(predictions))
            weighted_sum = sum(w * p for w, p in zip(weights, predictions))

            if weighted_sum > best_weighted_sum:
                best_weighted_sum = weighted_sum
                best_inputs = optimized_inputs

    if best_inputs:
        print("Best optimized inputs:", best_inputs)
        print("Best weighted sum of predictions:", best_weighted_sum)
    else:
        print("Optimization did not succeed in improving.")

# Example usage
inputs = [1000, 2000, 3000, 4000, 5]  # initial guesses for inputs, last one is fixed
model_directory = '/home/yui/Downloads/read_and_play/model_output'
optimize(inputs, model_directory)



Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 4 columns and 0 nonzeros
Model fingerprint: 0x785b59bd
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [2e+04, 2e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 23, time in user-callback 0.00 sec
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 11th Gen Intel(R) Core(TM) 

In [39]:
!pip install colorama==0.4.4
!pip install bayesian-optimization==1.4.0

import colorama
from bayes_opt import BayesianOptimization

import numpy as np
import torch
import joblib
import os



def black_box_function(battery_2, gas_cc, upv, wind_ons):
    inputs = [battery_2, gas_cc, upv, wind_ons, fixed_input]  # fixed_input is globally defined
    output_values = predictor.predict_quantiles(inputs)
    objective_value = sum(output_values)  # Example: sum of outputs as objective
    return objective_value

# Fixed last input variable
fixed_input = 10.0  # Adjust this value as necessary

# Define the model directory path correctly
model_directory = '/home/yui/Downloads/read_and_play/model_output'

# Bounded region of parameter space for the four variables being optimized
pbounds = {
    'battery_2': (57.542640, 12822.275981),
    'gas_cc': (20.763935, 12810.664546),
    'upv': (51.748227, 12833.113200),
    'wind_ons': (43.879052, 12630.725896)
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=2,
    n_iter=10,
)

print("Best result:", optimizer.max)


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
|   iter    |  target   | battery_2 |  gas_cc   |    upv    | wind_ons  |
-------------------------------------------------------------------------
| 1         | 2.255e+10 | 5.381e+03 | 9.234e+03 | 53.21     | 3.849e+03 |
| 2         | 2.256e+10 | 1.931e+03 | 1.202e+03 | 2.432e+03 | 4.393e+03 |
| 3         | 1.568e+10 | 156.1     | 2.091e+03 | 9.118e+03 | 9.67e+03  |
| 4         | 1.637e+10 | 5.707e+03 | 5.309e+03 | 7.053e+03 | 9.411e+03 |
| 5         | 1.456e+10 | 8.015e+03 | 2.876e+03 | 1.231e+04 | 1.017e+04 |
| 6         | 1.566e+10 | 328.2     | 1.035e+04 | 1.238e+04 | 8.881e+03 |
| 7         | 1.714e+10 | 4.709e+03 | 307.2     | 2e+03     | 1.227e+04 |
| 8         | 2.212e+10 | 1.788e+03 | 1.279e+03 | 2.749e+03 | 4.513e+03 |
| 9         | 3.577e+10 | 6.958e+03 | 1.186e+03 | 51.75     | 43.88     |
| 10        | 4.793e+10 | 1.28